<a href="https://colab.research.google.com/github/RohitGanji/CNN-White_Blood_Cancer_Classification-ALL_MM/blob/main/White%20Blood%20Cancer%20Classification%20using%20Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c1cd7f27-00c8-2248-bd3f-a3e92a1956ba)


In [2]:
import tensorflow as tf

train_dir = '/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/training'
test_dir = '/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/testing'

IMG_SIZE = (224, 224)

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.2,
                                                                 subset='training',
                                                                 seed=1302)

val_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE,
                                                                 validation_split=0.2,
                                                                 subset='validation',
                                                                 seed=1302)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                 label_mode="binary",
                                                                 image_size=IMG_SIZE)

Found 173 files belonging to 2 classes.
Using 139 files for training.
Found 173 files belonging to 2 classes.
Using 34 files for validation.
Found 29 files belonging to 2 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
], name = "data_augmentation")

In [4]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [5]:
# Create checkpoint callback
checkpoint_path = "/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [6]:
# Create a base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = True

for layer in base_model.layers[:-3]:
  layer.trainable = False

# Setup model architecture
inputs = layers.Input(shape=(224,224,3), name="input_shape")
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="GAP")(x)
x = layers.Dense(10, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy", f1_metric])

16711680/16705208 [==============================] - 0s 0us/step


In [7]:
# Fit the model
history = model.fit(train_data,
                    epochs=8,
                    validation_data=val_data,
                    validation_steps=int(len(val_data)),
                    callbacks=[checkpoint_callback])

Epoch 1/8
5/5 [==============================] - 126s 5s/step - loss: 0.4505 - accuracy: 0.8777 - f1_metric: 0.9007 - val_loss: 0.3169 - val_accuracy: 0.8529 - val_f1_metric: 0.4444
Epoch 2/8
5/5 [==============================] - 54s 623ms/step - loss: 0.1014 - accuracy: 1.0000 - f1_metric: 1.0000 - val_loss: 0.5263 - val_accuracy: 0.6471 - val_f1_metric: 0.6992
Epoch 3/8
5/5 [==============================] - 7s 613ms/step - loss: 0.0175 - accuracy: 1.0000 - f1_metric: 1.0000 - val_loss: 0.2196 - val_accuracy: 0.9118 - val_f1_metric: 0.4706
Epoch 4/8
5/5 [==============================] - 7s 730ms/step - loss: 0.0085 - accuracy: 1.0000 - f1_metric: 1.0000 - val_loss: 0.2006 - val_accuracy: 0.9412 - val_f1_metric: 0.9687
Epoch 5/8
5/5 [==============================] - 8s 765ms/step - loss: 0.0062 - accuracy: 1.0000 - f1_metric: 1.0000 - val_loss: 0.3876 - val_accuracy: 0.9118 - val_f1_metric: 0.4571
Epoch 6/8
5/5 [==============================] - 7s 613ms/step - loss: 0.0022 - accur

In [8]:
model.evaluate(val_data)

2/2 [==============================] - 1s 18ms/step - loss: 0.3131 - accuracy: 0.9118 - f1_metric: 0.8021


[0.3131120204925537, 0.9117646813392639, 0.8020832538604736]

In [9]:
model.load_weights('/content/drive/MyDrive/Datasets/SN-AM-BALL-MM/checkpoint')

In [10]:
model.evaluate(test_data)

1/1 [==============================] - 14s 14s/step - loss: 0.1629 - accuracy: 0.8966 - f1_metric: 0.9032


[0.16292905807495117, 0.8965517282485962, 0.9032257199287415]